<a href="https://colab.research.google.com/github/priyasi345/nlp_projects/blob/master/LSTM_Insincere%20Questions%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import os
import spacy
import string
import re
import numpy as np
from spacy.symbols import ORTH
from collections import Counter
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import warnings
warnings.filterwarnings('ignore')



In [ ]:

PATH = Path("/content/")
list(PATH.iterdir())

[PosixPath('/content/.config'),
 PosixPath('/content/train.csv'),
 PosixPath('/content/test.csv (1).zip'),
 PosixPath('/content/sample_submission (2).csv'),
 PosixPath('/content/sample_data')]

In [ ]:
!unzip '/content/test.csv (1).zip'

Archive:  /content/test.csv (1).zip
  inflating: test.csv                


In [ ]:
!unzip '/content/trainn.csv.zip'

unzip:  cannot find or open /content/trainn.csv.zip, /content/trainn.csv.zip.zip or /content/trainn.csv.zip.ZIP.


In [ ]:
df_train = pd.read_csv(PATH/"train.csv")
df_test = pd.read_csv(PATH/"test.csv")

In [ ]:
df_train.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0.0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0.0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0.0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0.0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0.0


In [ ]:
df_train['question_text'] = df_train['question_text'].astype(str)
df_train['qid'] = df_train['qid'].astype(str)

In [ ]:

re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
def sub_br(x): return re_br.sub("\n", x)

my_tok = spacy.load('en')
def spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(sub_br(x))]

In [ ]:
text = df_train['question_text'][0]
spacy_tok(text)

['How',
 'did',
 'Quebec',
 'nationalists',
 'see',
 'their',
 'province',
 'as',
 'a',
 'nation',
 'in',
 'the',
 '1960s',
 '?']

In [ ]:

counts = Counter()
for q in df_train['question_text']:
    counts.update(spacy_tok(q))

In [ ]:
for word in list(counts):
    if counts[word] < 5:
        del counts[word]

In [ ]:
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:

def encode_sentence(row, vocab2index, N=100, padding_start=True):
    words = spacy_tok(row)
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in words])
    l = min(N, len(enc1))
    if padding_start:
        enc[:l] = enc1[:l]
    else:
        enc[N-l:] = enc1[:l]
    return enc, l

In [ ]:
encode_sentence(df_train['question_text'][0], vocab2index, N=100, padding_start=False)

(array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15],
       dtype=int32), 14)

In [ ]:

x_train_len = np.array([len(q.split()) for q in df_train['question_text']])
x_test_len = np.array([len(q.split()) for q in df_test['question_text']])

In [ ]:
x_train_len.max(), x_test_len.max()

(66, 87)

In [ ]:
N_train = np.percentile(x_train_len, 99)
N_test = np.percentile(x_test_len, 99)

N_train, N_test

(39.0, 39.0)

In [ ]:
N=40


In [ ]:

train, valid = train_test_split(df_train, test_size=0.2)

In [ ]:
class QuoraDataset(Dataset):
    def __init__(self, df, vocab2index, is_test=False, N=40, padding_start=True):
        self.question = [encode_sentence(q, vocab2index, N, padding_start) for q in df['question_text']]
        self.is_test = is_test
        if self.is_test:
            self.y = None
        else:
            self.y = list(df["target"])

    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x, s = self.question[idx]
        
        if self.is_test:
            return x
        else:
            y = self.y[idx]
            return x, s, y

In [ ]:

train_ds = QuoraDataset(train, vocab2index, N=40, padding_start=False)
valid_ds = QuoraDataset(valid, vocab2index, N=40, padding_start=False)

In [ ]:

batch_size = 1000
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
x,s,y = next(iter(train_dl))

In [ ]:
train_ds[0]

(array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     2,    33, 14790,    59,
        14197,  4070,    17,    15], dtype=int32), 8, 0.0)

In [ ]:
!unzip '/content/1835_3176_compressed_glove.6B.100d.txt.zip'

Archive:  /content/1835_3176_compressed_glove.6B.100d.txt.zip
  inflating: glove.6B.100d.txt       


In [ ]:
def loadGloveModel(gloveFile="glove.6B.100d.txt"):
    """ Loads word vectors into a dictionary."""
    f = open(gloveFile,'r')
    word_vecs = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        word_vecs[word] = np.array([float(val) for val in splitLine[1:]])
    return word_vecs
word_vecs = loadGloveModel()

In [ ]:
def random_word_vector(D=90):
    """Create arandom word vector
    
    0.25 is chosen so the unknown vectors have (approximately) same variance 
    as pre-trained ones
    """
    return np.random.uniform(-0.25,0.25,D)



In [ ]:
def create_embedding_matrix(word_vecs, vocab2index, words, D=90):
    """Creates embedding matrix from word vectors. """
    V = len(words)
    W = np.zeros((V, D), dtype="float32")
    W[0] = np.zeros(D, dtype='float32')
    i = 1
    for i in range(1, V):
        if words[i] in word_vecs:
            W[i] = word_vecs[words[i]]
        else:
            W[i] = random_word_vector()
    return W

In [ ]:
embedding_matrix = create_embedding_matrix(word_vecs, vocab2index, words)
embedding_matrix.shape

ValueError: ignored

In [ ]:

class GRUBiModel(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim, embedding_matrix) :
        super(GRUBiModel,self).__init__()
        self.hidden_dim = hidden_dim

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.dropout = nn.Dropout(0.5)

        self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True,
                            dropout=0.3, bidirectional=True)
        self.linear = nn.Linear(2*hidden_dim, 1)
        
    def forward(self, x, s):
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True)
        out_pack, ht = self.gru(x_pack)
        h = torch.cat((ht[-2,:,:], ht[-1,:,:]), dim = 1)
        return self.linear(h)

In [ ]:
# Define the first NN 
class LSTM_GRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, embedding_matrix):
        super(LSTM_GRU, self).__init__()
        
        self.hidden_dim = hidden_dim

        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embeddings.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embeddings.weight.requires_grad = False ## freeze embeddings
        self.dropout = nn.Dropout(0.5)

#         self.gru = nn.GRU(embedding_dim, hidden_dim, num_layers=2, batch_first=True,
#                             dropout=0.3)
        
        
        self.embedding_dropout = nn.Dropout2d(0.01)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, bidirectional=True, batch_first=True)
        self.gru = nn.GRU(embedding_dim*2, hidden_dim, bidirectional=True, batch_first=True)
        
        self.linear = nn.Linear(2*hidden_dim, 1)
#         self.linear = nn.Linear(1024+features.shape[1], 16)
#         self.relu = nn.ELU()
#         self.dropout = nn.Dropout(0.01)
        
    def forward(self, x, s):
        s, sort_index = torch.sort(s, 0,descending=True)
        s = s.numpy().tolist()
        x = x[sort_index]
        x = self.embeddings(x)
        x = self.dropout(x)
#         x_pack = pack_padded_sequence(x, s, batch_first=True)
        lstm_pack, (ht, ct) = self.lstm(x)
        out_pack, ht = self.gru(lstm_pack)
        h = torch.cat((ht[-2,:,:], ht[-1,:,:]), dim = 1)
        return self.linear(h)
        

In [ ]:
def train_epocs(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)
    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, s, y in train_dl:
            x = x.long()
            y = y.float()
            y_pred = model(x, s)
            optimizer.zero_grad()
            loss = F.binary_cross_entropy_with_logits(y_pred, y.unsqueeze(1))
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc = val_metrics(model, valid_dl)
        if i % 5 == 1:
            print("train loss %.3f val loss %.3f and val accuracy %.3f" % (sum_loss/total, val_loss, val_acc))

def val_metrics(model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    for x, s, y in valid_dl:
        x = x.long()
        y = y.float().unsqueeze(1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct += (y_pred.float() == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
    return sum_loss/total, correct/total

In [ ]:

vocab_size = len(words)
print(vocab_size)

57673


In [ ]:
model = LSTM_GRU(vocab_size, 300, 300, embedding_matrix)

NameError: ignored

In [ ]:

%%time
train_epocs(model, epochs=15, lr=0.01)

In [ ]:
def save_model(m, p): 
    torch.save(m.state_dict(), p)
    
def load_model(m, p): 
    m.load_state_dict(torch.load(p))

In [ ]:
p1 = "./models/model-96.pth"
save_model(model, p1)


In [ ]:
model1 = LSTMModel(vocab_size, 50, 50)
model_path = "./models/model-96.pth"
load_model(model1, model_path)

In [ ]:
test_ds = QuoraDataset(df_test, vocab2index, is_test=True, N=40, padding_start=False)
test_dl = DataLoader(test_ds, batch_size=1000)

In [ ]:
df_submission = pd.read_csv(PATH/"sample_submission.csv")

In [ ]:
preds=[]
for x in test_dl:
    x = x[0].long()
    out = model(x)
    pred = (out > 0.0).long()
    preds.append(pred.numpy())

In [ ]:
df_submission['prediction'] = np.vstack(preds)

In [ ]:
df_submission.head()

In [ ]:
df_submission.to_csv('submission.csv', index=False)